In [0]:
# Este código se ejecuta en una celda de un Notebook de Databricks
# Su responsabilidad es validar, limpiar y estandarizar la tabla de stock,
# guardando solo las columnas 'codigo_producto' y 'stock' en la Capa de Plata.
from pyspark.sql.functions import col, when, upper, regexp_extract
from pyspark.sql.types import IntegerType

# 1. --- Configuración de Tablas ---
source_table = "workspace.tecnomundo_data_raw.stock_raw"
output_silver_table = "workspace.tecnomundo_data_processed.stock_cleaned"
output_quarantine_table = "workspace.tecnomundo_data_reporting.stock_silver_quarantine"

print(f"Tabla de Origen (Bronce): {source_table}")
print(f"Tabla de Destino (Plata): {output_silver_table}")
print(f"Tabla de Cuarentena (Errores): {output_quarantine_table}")


# 2. --- Lectura de la Capa de Bronce ---
print(f"\nLeyendo datos desde '{source_table}'...")
try:
    df_bronze = spark.table(source_table)
    print("Lectura exitosa.")
except Exception as e:
    print(f"Error al leer la tabla de origen: {e}")
    dbutils.notebook.exit("No se pudo leer la tabla de origen.")


# 3. --- Validación y Separación de Datos ---
print("\nValidando y separando registros limpios de registros con errores...")

# Añadimos una columna temporal para convertir el stock a entero.
# Los valores no numéricos se convertirán en NULL.
df_pre_validated = df_bronze.withColumn("stock_int", col("stock").cast(IntegerType()))

# Definimos las reglas de calidad:
# 1. El 'codigo_producto' no debe ser nulo o estar vacío.
# 2. El 'stock' debe ser un número (no nulo después del cast) y mayor o igual a -1.
df_with_validation = df_pre_validated.withColumn(
    "is_valid",
    (col("codigo_producto").isNotNull() & (col("codigo_producto") != "")) &
    (col("stock_int").isNotNull() & (col("stock_int") >= -1))
)

# Dividimos el DataFrame en dos: uno para los datos válidos y otro para la cuarentena.
df_valid = df_with_validation.filter(col("is_valid") == True)
df_quarantine = df_with_validation.filter(col("is_valid") == False).drop("is_valid", "stock_int")

valid_count = df_valid.count()
quarantine_count = df_quarantine.count()

print(f"Se encontraron {valid_count} registros válidos (stock >= -1).")
print(f"Se encontraron {quarantine_count} registros para cuarentena (stock < -1 o no numérico).")


# 4. --- Procesamiento de Datos Válidos (Capa de Plata) ---
print("\nProcesando, seleccionando y guardando datos limpios en la Capa de Plata...")

# Aplicamos las transformaciones y seleccionamos ÚNICAMENTE las columnas finales.
df_silver = df_valid.select(
    # Estandarizamos 'codigo_producto'
    upper(
        when(
            col("codigo_producto").rlike("^[A-Z]\\d{2}-"), 
            regexp_extract(col("codigo_producto"), r"-(.*)$", 1)
        ).otherwise(col("codigo_producto"))
    ).alias("codigo_producto"),
    
    # Usamos la columna ya convertida a entero y la renombramos a 'stock'.
    col("stock_int").alias("stock")
)

# Guardamos el DataFrame final y limpio en la capa de Plata.
(df_silver.write
 .mode("overwrite")
 .option("overwriteSchema", "true")
 .saveAsTable(output_silver_table))

print(f"¡Tabla de stock limpia y con esquema final guardada exitosamente en '{output_silver_table}'!")
print("Esquema final de la Capa de Plata:")
df_silver.printSchema()


# 5. --- Manejo de Datos en Cuarentena ---
if quarantine_count > 0:
    print(f"\n--- INICIO: Filas con Errores Detectados ({quarantine_count}) ---")
    df_quarantine.show(truncate=False)
    print("--- FIN: Filas con Errores Detectados ---")
    
    print(f"\nGuardando registros con errores en la tabla de cuarentena '{output_quarantine_table}'...")
    # Guardamos los registros originales que fallaron la validación (con todas sus columnas)
    (df_quarantine.write
     .mode("overwrite")
     .option("overwriteSchema", "true")
     .saveAsTable(output_quarantine_table))
    print("¡Registros en cuarentena guardados!")
else:
    print("\nNo se encontraron registros para la cuarentena. ¡Todos los datos pasaron la validación!")


# 6. --- Verificación Final ---
print("\nMostrando una muestra de los datos limpios (Capa de Plata):")
display(spark.table(output_silver_table))

if quarantine_count > 0:
    print("\nMostrando una muestra de los datos en cuarentena:")
    display(spark.table(output_quarantine_table))


In [0]:
display(spark.table("workspace.tecnomundo_data_reporting.stock_silver_quarantine"))